# Full Accessibility Tree Extraction

This notebook demonstrates how to fetch and parse the complete accessibility tree with all available data.

In [1]:
import subprocess
import json
from typing import Dict, List, Any

## Fetch Full Accessibility Tree

In [2]:
def get_full_a11y_tree() -> Dict[str, Any]:
    """
    Fetch the complete accessibility tree from the device.
    Returns the parsed tree as a Python dictionary.
    """
    result = subprocess.run(
        ['adb', 'shell', 'content', 'query', '--uri', 'content://com.droidrun.portal/a11y_tree_full'],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        raise Exception(f"ADB command failed: {result.stderr}")
    
    # Parse the content provider response
    # Format: Row: 0 result={"status":"success","data":"..."}
    for line in result.stdout.strip().split('\n'):
        if 'result=' in line:
            json_str = line.split('result=', 1)[1]
            outer_json = json.loads(json_str)
            
            if outer_json.get('status') != 'success':
                raise Exception(f"Error from service: {outer_json.get('error')}")
            
            # Parse the inner data (which is a JSON string)
            tree = json.loads(outer_json['data'])
            return tree
    
    raise Exception("No result found in ADB output")

# Fetch the tree
tree = get_full_a11y_tree()
print(f"Successfully fetched accessibility tree!")
print(f"Root element className: {tree.get('className')}")
print(f"Root element packageName: {tree.get('packageName')}")

Successfully fetched accessibility tree!
Root element className: android.widget.FrameLayout
Root element packageName: com.reddit.frontpage


## Explore Tree Structure

In [3]:
def count_nodes(node: Dict[str, Any]) -> int:
    """Recursively count all nodes in the tree."""
    count = 1
    for child in node.get('children', []):
        count += count_nodes(child)
    return count

def get_max_depth(node: Dict[str, Any], depth: int = 0) -> int:
    """Get the maximum depth of the tree."""
    if not node.get('children'):
        return depth
    return max(get_max_depth(child, depth + 1) for child in node['children'])

total_nodes = count_nodes(tree)
max_depth = get_max_depth(tree)

print(f"Total nodes: {total_nodes}")
print(f"Maximum depth: {max_depth}")
print(f"Children of root: {tree.get('childCount', 0)}")

Total nodes: 243
Maximum depth: 25
Children of root: 2


## Inspect Available Fields

In [4]:
# Show all available fields in the root node
print("All fields in root node:")
for key in sorted(tree.keys()):
    value = tree[key]
    if key == 'children':
        print(f"  {key}: [{len(value)} children]")
    elif isinstance(value, dict):
        print(f"  {key}: {value}")
    elif isinstance(value, list):
        print(f"  {key}: [{len(value)} items]")
    elif isinstance(value, str) and len(value) > 50:
        print(f"  {key}: '{value[:50]}...'")
    else:
        print(f"  {key}: {value}")

All fields in root node:
  actionCount: 4
  actionList: [4 items]
  boundsInParent: {'left': 0, 'top': 0, 'right': 1080, 'bottom': 2400}
  boundsInScreen: {'left': 0, 'top': 0, 'right': 1080, 'bottom': 2400}
  childCount: 2
  children: [2 children]
  className: android.widget.FrameLayout
  contentDescription: 
  drawingOrder: 0
  error: 
  hint: 
  inputType: 0
  isAccessibilityFocused: False
  isCheckable: False
  isChecked: False
  isClickable: False
  isContextClickable: False
  isDismissable: False
  isEditable: False
  isEnabled: True
  isFocusable: False
  isFocused: False
  isHeading: False
  isImportantForAccessibility: True
  isLongClickable: False
  isMultiLine: False
  isPassword: False
  isScreenReaderFocusable: False
  isScrollable: False
  isSelected: False
  isShowingHintText: False
  isTextSelectable: False
  isVisibleToUser: True
  liveRegion: 0
  maxTextLength: -1
  movementGranularities: 0
  packageName: com.reddit.frontpage
  paneTitle: 
  resourceId: 
  stateDescri

## Find Clickable Elements

In [5]:
def find_clickable_elements(node: Dict[str, Any], results: List[Dict] = None) -> List[Dict]:
    """Find all clickable elements in the tree."""
    if results is None:
        results = []
    
    if node.get('isClickable'):
        results.append({
            'resourceId': node.get('resourceId', ''),
            'className': node.get('className', ''),
            'text': node.get('text', ''),
            'contentDescription': node.get('contentDescription', ''),
            'bounds': node.get('boundsInScreen', {}),
        })
    
    for child in node.get('children', []):
        find_clickable_elements(child, results)
    
    return results

clickable = find_clickable_elements(tree)
print(f"Found {len(clickable)} clickable elements:\n")

for i, elem in enumerate(clickable[:10], 1):  # Show first 10
    print(f"{i}. {elem['className']}")
    if elem['text']:
        print(f"   Text: {elem['text']}")
    if elem['resourceId']:
        print(f"   ID: {elem['resourceId']}")
    if elem['contentDescription']:
        print(f"   Description: {elem['contentDescription']}")
    print(f"   Bounds: {elem['bounds']}")
    print()

Found 43 clickable elements:

1. android.widget.TextView
   Text: r/AmItheAsshole
   ID: post_subreddit
   Bounds: {'left': -948, 'top': 230, 'right': -674, 'bottom': 360}

2. android.view.View
   Description: Community Status is the CCA emoji
   Bounds: {'left': -690, 'top': 230, 'right': -558, 'bottom': 360}

3. android.widget.TextView
   Text: Do you have a butt? Read this.
   ID: post_title
   Bounds: {'left': -1036, 'top': 360, 'right': -399, 'bottom': 492}

4. android.view.View
   Bounds: {'left': -1036, 'top': 445, 'right': -895, 'bottom': 574}

5. android.view.View
   ID: add_comment_button
   Bounds: {'left': -1058, 'top': 2219, 'right': -44, 'bottom': 2351}

6. android.view.View
   Bounds: {'left': -1068, 'top': 92, 'right': -936, 'bottom': 224}

7. android.view.View
   Bounds: {'left': -925, 'top': 91, 'right': -603, 'bottom': 223}

8. android.view.View
   Bounds: {'left': -493, 'top': 92, 'right': -376, 'bottom': 224}

9. android.view.View
   Bounds: {'left': -376, 'top': 9

## Find Editable Elements (Input Fields)

In [6]:
def find_editable_elements(node: Dict[str, Any], results: List[Dict] = None) -> List[Dict]:
    """Find all editable/input elements in the tree."""
    if results is None:
        results = []
    
    if node.get('isEditable'):
        results.append({
            'resourceId': node.get('resourceId', ''),
            'className': node.get('className', ''),
            'text': node.get('text', ''),
            'hint': node.get('hint', ''),
            'isPassword': node.get('isPassword', False),
            'inputType': node.get('inputType', 0),
            'bounds': node.get('boundsInScreen', {}),
        })
    
    for child in node.get('children', []):
        find_editable_elements(child, results)
    
    return results

editable = find_editable_elements(tree)
print(f"Found {len(editable)} editable elements:\n")

for i, elem in enumerate(editable, 1):
    print(f"{i}. {elem['className']}")
    if elem['text']:
        print(f"   Text: {elem['text']}")
    if elem['hint']:
        print(f"   Hint: {elem['hint']}")
    if elem['isPassword']:
        print(f"   ⚠️  Password field")
    print(f"   Input Type: {elem['inputType']}")
    print(f"   Bounds: {elem['bounds']}")
    print()

Found 0 editable elements:



## Find Scrollable Containers

In [7]:
def find_scrollable_elements(node: Dict[str, Any], results: List[Dict] = None) -> List[Dict]:
    """Find all scrollable containers."""
    if results is None:
        results = []
    
    if node.get('isScrollable'):
        results.append({
            'resourceId': node.get('resourceId', ''),
            'className': node.get('className', ''),
            'bounds': node.get('boundsInScreen', {}),
            'childCount': node.get('childCount', 0),
            'collectionInfo': node.get('collectionInfo'),
        })
    
    for child in node.get('children', []):
        find_scrollable_elements(child, results)
    
    return results

scrollable = find_scrollable_elements(tree)
print(f"Found {len(scrollable)} scrollable containers:\n")

for i, elem in enumerate(scrollable, 1):
    print(f"{i}. {elem['className']}")
    if elem['resourceId']:
        print(f"   ID: {elem['resourceId']}")
    print(f"   Children: {elem['childCount']}")
    if elem['collectionInfo']:
        info = elem['collectionInfo']
        print(f"   Collection: {info.get('rowCount')}x{info.get('columnCount')} grid")
    print(f"   Bounds: {elem['bounds']}")
    print()

Found 4 scrollable containers:

1. androidx.viewpager.widget.ViewPager
   ID: com.reddit.frontpage:id/fragment_pager
   Children: 3
   Bounds: {'left': 0, 'top': 0, 'right': 1080, 'bottom': 2400}

2. android.view.View
   ID: post_detail_lazy_column
   Children: 1
   Collection: -1x1 grid
   Bounds: {'left': -1080, 'top': 235, 'right': 0, 'bottom': 2214}

3. android.view.View
   ID: post_detail_lazy_column
   Children: 1
   Collection: -1x1 grid
   Bounds: {'left': 0, 'top': 235, 'right': 1080, 'bottom': 2214}

4. android.view.View
   ID: post_detail_lazy_column
   Children: 3
   Collection: -1x1 grid
   Bounds: {'left': 1080, 'top': 235, 'right': 2160, 'bottom': 2397}



## Visualize Tree Structure (First Few Levels)

In [8]:
def print_tree(node: Dict[str, Any], depth: int = 0, max_depth: int = 3):
    """Print a visual representation of the tree structure."""
    if depth > max_depth:
        return
    
    indent = "  " * depth
    
    # Build node description
    desc = node.get('className', 'Unknown')
    if node.get('text'):
        desc += f" [text: '{node['text'][:30]}']" if len(node.get('text', '')) > 30 else f" [text: '{node['text']}']" 
    if node.get('resourceId'):
        desc += f" [id: {node['resourceId']}]"
    
    # Add important flags
    flags = []
    if node.get('isClickable'):
        flags.append('clickable')
    if node.get('isEditable'):
        flags.append('editable')
    if node.get('isScrollable'):
        flags.append('scrollable')
    if flags:
        desc += f" ({', '.join(flags)})"
    
    print(f"{indent}├─ {desc}")
    
    # Recurse for children
    for child in node.get('children', []):
        print_tree(child, depth + 1, max_depth)

print("Tree structure (first 3 levels):\n")
print_tree(tree)

Tree structure (first 3 levels):

├─ android.widget.FrameLayout
  ├─ android.view.ViewGroup
    ├─ android.view.View
      ├─ android.view.View [id: toast_container]
  ├─ androidx.drawerlayout.widget.DrawerLayout [id: com.reddit.frontpage:id/drawer_layout]
    ├─ android.view.ViewGroup
      ├─ android.view.ViewGroup


## Export Tree to JSON File

In [9]:
# Save the full tree to a JSON file for further analysis
output_file = 'accessibility_tree.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(tree, f, indent=2, ensure_ascii=False)

print(f"✓ Accessibility tree saved to: {output_file}")
print(f"  File size: {len(json.dumps(tree))} bytes")

✓ Accessibility tree saved to: accessibility_tree.json
  File size: 332904 bytes


## Search Elements by Criteria

In [33]:
def search_elements(node: Dict[str, Any], criteria: Dict[str, Any], results: List[Dict] = None) -> List[Dict]:
    """
    Search for elements matching the given criteria.
    
    Example criteria:
    - {'text': 'Login'}
    - {'className': 'Button', 'isEnabled': True}
    - {'resourceId': 'com.example:id/search_button'}
    """
    if results is None:
        results = []
    
    # Check if node matches all criteria
    matches = all(
        str(node.get(key, '')).lower().find(str(value).lower()) >= 0 
        if isinstance(value, str) 
        else node.get(key) == value
        for key, value in criteria.items()
    )
    
    if matches:
        results.append(node)
    
    for child in node.get('children', []):
        search_elements(child, criteria, results)
    
    return results

# Example searches
print("Example searches:\n")

# Find all buttons
buttons = search_elements(tree, {'className': 'Button'})
print(f"Buttons found: {len(buttons)}")

# Find elements with specific text
elements_with_text = search_elements(tree, {'text': 'Search'})
print(f"Elements with 'Search' text: {len(elements_with_text)}")

# Find enabled checkboxes
checkboxes = search_elements(tree, {'isCheckable': True, 'isEnabled': True})
print(f"Enabled checkboxes: {len(checkboxes)}")

Example searches:

Buttons found: 0
Elements with 'Search' text: 0
Enabled checkboxes: 2


## Statistics Summary

In [34]:
def gather_stats(node: Dict[str, Any], stats: Dict = None) -> Dict:
    """Gather statistics about the accessibility tree."""
    if stats is None:
        stats = {
            'total_nodes': 0,
            'clickable': 0,
            'editable': 0,
            'scrollable': 0,
            'checkable': 0,
            'enabled': 0,
            'visible': 0,
            'with_text': 0,
            'with_description': 0,
            'classes': {},
        }
    
    stats['total_nodes'] += 1
    
    if node.get('isClickable'):
        stats['clickable'] += 1
    if node.get('isEditable'):
        stats['editable'] += 1
    if node.get('isScrollable'):
        stats['scrollable'] += 1
    if node.get('isCheckable'):
        stats['checkable'] += 1
    if node.get('isEnabled'):
        stats['enabled'] += 1
    if node.get('isVisibleToUser'):
        stats['visible'] += 1
    if node.get('text'):
        stats['with_text'] += 1
    if node.get('contentDescription'):
        stats['with_description'] += 1
    
    # Count class names
    class_name = node.get('className', 'Unknown')
    stats['classes'][class_name] = stats['classes'].get(class_name, 0) + 1
    
    for child in node.get('children', []):
        gather_stats(child, stats)
    
    return stats

stats = gather_stats(tree)

print("=" * 60)
print("ACCESSIBILITY TREE STATISTICS")
print("=" * 60)
print(f"\nTotal nodes: {stats['total_nodes']}")
print(f"\nInteractive elements:")
print(f"  Clickable: {stats['clickable']}")
print(f"  Editable: {stats['editable']}")
print(f"  Scrollable: {stats['scrollable']}")
print(f"  Checkable: {stats['checkable']}")
print(f"\nState:")
print(f"  Enabled: {stats['enabled']}")
print(f"  Visible: {stats['visible']}")
print(f"\nContent:")
print(f"  With text: {stats['with_text']}")
print(f"  With description: {stats['with_description']}")
print(f"\nTop 10 most common classes:")
for class_name, count in sorted(stats['classes'].items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  {class_name}: {count}")

ACCESSIBILITY TREE STATISTICS

Total nodes: 18

Interactive elements:
  Clickable: 5
  Editable: 0
  Scrollable: 0
  Checkable: 2

State:
  Enabled: 18
  Visible: 18

Content:
  With text: 7
  With description: 3

Top 10 most common classes:
  android.widget.TextView: 7
  android.widget.LinearLayout: 4
  android.widget.FrameLayout: 3
  android.widget.Switch: 2
  android.view.ViewGroup: 1
  androidx.recyclerview.widget.RecyclerView: 1


## Filter Tree Like `a11y_tree` Endpoint

The `a11y_tree` endpoint applies filtering to show only relevant elements. This replicates the same filtering logic from `findAllVisibleElements()` in DroidrunAccessibilityService.kt:

1. **Screen bounds check**: Element must be within screen boundaries
2. **Minimum size check**: Element width > 5px AND height > 5px

Only elements that pass BOTH checks are included in the filtered tree.

In [35]:
def rects_intersect(rect1: Dict, rect2: Dict) -> bool:
    """Check if two rectangles intersect (like Rect.intersects in Android)."""
    return not (rect1['right'] <= rect2['left'] or
                rect1['left'] >= rect2['right'] or
                rect1['bottom'] <= rect2['top'] or
                rect1['top'] >= rect2['bottom'])

def filter_visible_elements(node: Dict[str, Any], screen_bounds: Dict, 
                            min_element_size: int = 5,
                            index_counter: Dict[str, int] = None) -> Dict[str, Any]:
    """
    Filter accessibility tree to include only visible, properly-sized elements.
    This replicates the logic from findAllVisibleElements() in DroidrunAccessibilityService.kt
    
    Args:
        node: The accessibility node to process
        screen_bounds: Screen boundaries dict with left, top, right, bottom
        min_element_size: Minimum width/height for an element (default 5px)
        index_counter: Mutable dict to track overlay index (starts at 1)
    
    Returns:
        Filtered node dict with only visible elements, or None if filtered out
    """
    if index_counter is None:
        index_counter = {'current': 1}
    
    # Get node bounds
    rect = node.get('boundsInScreen', {})
    
    # Check if element is in screen and has minimum size
    is_in_screen = rects_intersect(rect, screen_bounds)
    has_size = (rect.get('right', 0) - rect.get('left', 0) > min_element_size and 
                rect.get('bottom', 0) - rect.get('top', 0) > min_element_size)
    
    current_element = None
    
    # Only include element if it passes both checks
    if is_in_screen and has_size:
        # Create filtered element
        current_element = {
            'overlayIndex': index_counter['current'],
            'resourceId': node.get('resourceId', ''),
            'className': node.get('className', '').split('.')[-1],  # Short name
            'text': node.get('text', ''),
            'contentDescription': node.get('contentDescription', ''),
            'bounds': rect,
            'isClickable': node.get('isClickable', False),
            'isCheckable': node.get('isCheckable', False),
            'isEditable': node.get('isEditable', False),
            'isScrollable': node.get('isScrollable', False),
            'children': []
        }
        
        # Increment index counter
        index_counter['current'] += 1
    
    # Process children recursively
    filtered_children = []
    for child in node.get('children', []):
        filtered_child = filter_visible_elements(child, screen_bounds, min_element_size, index_counter)
        if filtered_child is not None:
            filtered_children.append(filtered_child)
    
    # Add children to current element or return only children if current element was filtered out
    if current_element is not None:
        current_element['children'] = filtered_children
        return current_element
    else:
        # If current element is filtered out but has filtered children, 
        # return them as siblings (flatten the tree)
        # Actually, looking at the Kotlin code more carefully, filtered-out parents
        # still pass their children along but the parent itself is not included
        # However, we need to return a single node or None, so we'll return None
        # and let the children be processed
        return None if not filtered_children else filtered_children[0] if len(filtered_children) == 1 else {
            'overlayIndex': 0,  # Virtual container
            'children': filtered_children,
            'resourceId': '',
            'className': 'VirtualContainer',
            'text': '',
            'contentDescription': '',
            'bounds': rect,
            'isClickable': False,
            'isCheckable': False,
            'isEditable': False,
            'isScrollable': False
        }

# Get screen bounds from root element
screen_bounds = tree.get('boundsInScreen', {})

print(f"Screen bounds: {screen_bounds}")
print(f"Screen size: {screen_bounds['right']}x{screen_bounds['bottom']}")

# Filter the tree
filtered_tree = filter_visible_elements(tree, screen_bounds)

# Count filtered nodes
def count_filtered_nodes(node):
    if node is None:
        return 0
    count = 1
    for child in node.get('children', []):
        count += count_filtered_nodes(child)
    return count

filtered_count = count_filtered_nodes(filtered_tree)

print(f"\nFiltered tree statistics:")
print(f"  Original nodes: {count_nodes(tree)}")
print(f"  Filtered nodes: {filtered_count}")
print(f"  Reduction: {count_nodes(tree) - filtered_count} nodes ({100 * (1 - filtered_count/count_nodes(tree)):.1f}% filtered out)")

Screen bounds: {'left': 0, 'top': 0, 'right': 1080, 'bottom': 2400}
Screen size: 1080x2400

Filtered tree statistics:
  Original nodes: 18
  Filtered nodes: 18
  Reduction: 0 nodes (0.0% filtered out)


## Explore Filtered Elements

In [36]:
def find_clickable_in_filtered(node: Dict[str, Any], results: List[Dict] = None) -> List[Dict]:
    """Find all clickable elements in the filtered tree."""
    if results is None:
        results = []
    
    if node is None:
        return results
    
    if node.get('isClickable'):
        results.append({
            'overlayIndex': node.get('overlayIndex'),
            'resourceId': node.get('resourceId', ''),
            'className': node.get('className', ''),
            'text': node.get('text', ''),
            'contentDescription': node.get('contentDescription', ''),
            'bounds': node.get('bounds', {}),
        })
    
    for child in node.get('children', []):
        find_clickable_in_filtered(child, results)
    
    return results

clickable_filtered = find_clickable_in_filtered(filtered_tree)
print(f"Found {len(clickable_filtered)} clickable elements in filtered tree (vs {len(clickable)} in full tree):\n")

for i, elem in enumerate(clickable_filtered[:10], 1):  # Show first 10
    print(f"{i}. [Index {elem['overlayIndex']}] {elem['className']}")
    if elem['text']:
        print(f"   Text: {elem['text']}")
    if elem['resourceId']:
        print(f"   ID: {elem['resourceId']}")
    if elem['contentDescription']:
        print(f"   Description: {elem['contentDescription']}")
    bounds = elem['bounds']
    print(f"   Bounds: ({bounds['left']}, {bounds['top']}) - ({bounds['right']}, {bounds['bottom']})")
    print(f"   Size: {bounds['right'] - bounds['left']}x{bounds['bottom'] - bounds['top']}")
    print()

Found 5 clickable elements in filtered tree (vs 5 in full tree):

1. [Index 2] FrameLayout
   ID: com.android.settings:id/action_bar_container
   Bounds: (0, 0) - (1080, 408)
   Size: 1080x408

2. [Index 4] FrameLayout
   ID: com.android.settings:id/up
   Description: Back
   Bounds: (56, 102) - (168, 214)
   Size: 112x112

3. [Index 7] LinearLayout
   ID: com.android.settings:id/main_switch_bar
   Bounds: (0, 428) - (1080, 630)
   Size: 1080x202

4. [Index 12] LinearLayout
   ID: com.android.settings:id/main_frame
   Bounds: (78, 821) - (839, 1039)
   Size: 761x218

5. [Index 15] Switch
   ID: com.android.settings:id/switchWidget
   Description: Shortcut settings
   Bounds: (864, 884) - (1002, 975)
   Size: 138x91



## Visualize Filtered Tree Structure

In [37]:
def print_filtered_tree(node: Dict[str, Any], depth: int = 0, max_depth: int = 3):
    """Print a visual representation of the filtered tree structure."""
    if node is None or depth > max_depth:
        return
    
    indent = "  " * depth
    
    # Build node description
    desc = f"[{node.get('overlayIndex', '?')}] {node.get('className', 'Unknown')}"
    if node.get('text'):
        text = node['text'][:30]
        desc += f" [text: '{text}']" if len(node.get('text', '')) <= 30 else f" [text: '{text}...']"
    if node.get('resourceId'):
        desc += f" [id: {node['resourceId']}]"
    
    # Add important flags
    flags = []
    if node.get('isClickable'):
        flags.append('clickable')
    if node.get('isEditable'):
        flags.append('editable')
    if node.get('isScrollable'):
        flags.append('scrollable')
    if flags:
        desc += f" ({', '.join(flags)})"
    
    print(f"{indent}├─ {desc}")
    
    # Recurse for children
    for child in node.get('children', []):
        print_filtered_tree(child, depth + 1, max_depth)

print("Filtered tree structure (first 3 levels):\n")
print_filtered_tree(filtered_tree)

print(f"\n\nFiltered tree max depth: {get_max_depth(filtered_tree) if filtered_tree else 0}")

Filtered tree structure (first 3 levels):

├─ [1] FrameLayout
  ├─ [2] FrameLayout [id: com.android.settings:id/action_bar_container] (clickable)
    ├─ [3] ViewGroup [id: com.android.settings:id/action_bar]
      ├─ [4] FrameLayout [id: com.android.settings:id/up] (clickable)
      ├─ [5] TextView [text: 'Droidrun Portal'] [id: com.android.settings:id/action_bar_title_expand]
  ├─ [6] RecyclerView [id: com.android.settings:id/recycler_view]
    ├─ [7] LinearLayout [id: com.android.settings:id/main_switch_bar] (clickable)
      ├─ [8] TextView [text: 'Use Droidrun Portal'] [id: com.android.settings:id/switch_text]
      ├─ [9] Switch [id: android:id/switch_widget]
    ├─ [10] TextView [text: 'Options'] [id: android:id/title]
    ├─ [11] LinearLayout
      ├─ [12] LinearLayout [id: com.android.settings:id/main_frame] (clickable)
      ├─ [15] Switch [id: com.android.settings:id/switchWidget] (clickable)
    ├─ [16] TextView [text: 'About Droidrun Portal'] [id: android:id/title]
    ├─ [

## Validate Against Actual `a11y_tree` Endpoint

Compare our filtered tree with the actual output from the `a11y_tree` endpoint to ensure the filtering logic matches.

In [38]:
def get_a11y_tree() -> List[Dict]:
    """Fetch the filtered accessibility tree from the a11y_tree endpoint."""
    result = subprocess.run(
        ['adb', 'shell', 'content', 'query', '--uri', 'content://com.droidrun.portal/a11y_tree'],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        raise Exception(f"ADB command failed: {result.stderr}")
    
    for line in result.stdout.strip().split('\n'):
        if 'result=' in line:
            json_str = line.split('result=', 1)[1]
            outer_json = json.loads(json_str)
            
            if outer_json.get('status') != 'success':
                raise Exception(f"Error from service: {outer_json.get('error')}")
            
            # Parse the inner data (which is a JSON string)
            tree_list = json.loads(outer_json['data'])
            return tree_list
    
    raise Exception("No result found in ADB output")

# Fetch actual filtered tree from endpoint
actual_tree = get_a11y_tree()

print("Actual a11y_tree endpoint response:")
print(f"  Type: {type(actual_tree)}")
print(f"  Number of root elements: {len(actual_tree) if isinstance(actual_tree, list) else 1}")

if isinstance(actual_tree, list) and len(actual_tree) > 0:
    # Count total elements
    def count_a11y_elements(elements):
        count = 0
        if isinstance(elements, list):
            for elem in elements:
                count += 1
                count += count_a11y_elements(elem.get('children', []))
        elif isinstance(elements, dict):
            count = 1
            count += count_a11y_elements(elements.get('children', []))
        return count
    
    actual_count = count_a11y_elements(actual_tree)
    
    print(f"  Total elements: {actual_count}")
    print(f"\nOur filtered tree count: {filtered_count}")
    print(f"Match: {'✓ YES' if actual_count == filtered_count else '✗ NO (difference: ' + str(abs(actual_count - filtered_count)) + ')'}")
    
    print(f"\nFirst element from a11y_tree endpoint:")
    first_elem = actual_tree[0]
    print(f"  Index: {first_elem.get('index')}")
    print(f"  Class: {first_elem.get('className')}")
    print(f"  Text: {first_elem.get('text')}")
    print(f"  Resource ID: {first_elem.get('resourceId')}")
    print(f"  Bounds: {first_elem.get('bounds')}")
    print(f"  Children: {len(first_elem.get('children', []))}")

Actual a11y_tree endpoint response:
  Type: <class 'list'>
  Number of root elements: 1
  Total elements: 18

Our filtered tree count: 18
Match: ✓ YES

First element from a11y_tree endpoint:
  Index: 1
  Class: FrameLayout
  Text: FrameLayout
  Resource ID: 
  Bounds: 0, 0, 1080, 2400
  Children: 2


In [41]:
def format_ui_elements(ui_data: List[Dict[str, Any]], level: int = 0) -> str:
    """
    Format UI elements in the exact format: index. className: "resourceId", "text" - (bounds)

    Args:
        ui_data: List of UI element dictionaries (can be from a11y_tree or filtered_tree)
        level: Indentation level for nested elements

    Returns:
        Formatted UI elements text
    """
    if not ui_data:
        return ""

    formatted_lines = []
    indent = "  " * level  # Indentation for nested elements

    # Handle both list and single element
    elements = ui_data if isinstance(ui_data, list) else [ui_data]

    for element in elements:
        if not isinstance(element, dict):
            continue

        # Extract element properties - support both 'index' (from a11y_tree) and 'overlayIndex' (from filtered_tree)
        index = element.get("index") or element.get("overlayIndex", "")
        class_name = element.get("className", "")
        resource_id = element.get("resourceId", "")
        text = element.get("text", "")
        bounds = element.get("bounds", "")
        children = element.get("children", [])

        # Format bounds - handle both string format "x1, y1, x2, y2" and dict format
        if isinstance(bounds, dict):
            bounds_str = f"{bounds.get('left', 0)}, {bounds.get('top', 0)}, {bounds.get('right', 0)}, {bounds.get('bottom', 0)}"
        else:
            bounds_str = bounds

        # Format the line: index. className: "resourceId", "text" - (bounds)
        line_parts = []
        if index != "":
            line_parts.append(f"{index}.")
        if class_name:
            line_parts.append(class_name + ":")

        # Build the quoted details section
        details = []
        if resource_id:
            details.append(f'"{resource_id}"')
        if text:
            details.append(f'"{text}"')

        if details:
            line_parts.append(", ".join(details))

        if bounds_str:
            line_parts.append(f"- ({bounds_str})")

        formatted_line = f"{indent}{' '.join(line_parts)}"
        formatted_lines.append(formatted_line)

        # Recursively format children with increased indentation
        if children:
            child_formatted = format_ui_elements(children, level + 1)
            if child_formatted:
                formatted_lines.append(child_formatted)

    return "\n".join(formatted_lines)

# Test with actual a11y_tree endpoint data
print("Formatted a11y_tree (from endpoint):")
print(format_ui_elements(actual_tree))
print("\n" + "="*60 + "\n")

# Test with our filtered tree
print("Formatted filtered_tree (our implementation):")
print(format_ui_elements([filtered_tree]))

Formatted a11y_tree (from endpoint):
1. FrameLayout: "FrameLayout" - (0, 0, 1080, 2400)
  2. FrameLayout: "com.android.settings:id/action_bar_container", "action_bar_container" - (0, 0, 1080, 408)
    3. ViewGroup: "com.android.settings:id/action_bar", "action_bar" - (0, 80, 1080, 408)
      4. FrameLayout: "com.android.settings:id/up", "Back" - (56, 102, 168, 214)
      5. TextView: "com.android.settings:id/action_bar_title_expand", "Droidrun Portal" - (72, 248, 724, 380)
  6. RecyclerView: "com.android.settings:id/recycler_view", "recycler_view" - (0, 408, 1080, 2400)
    7. LinearLayout: "com.android.settings:id/main_switch_bar", "main_switch_bar" - (0, 428, 1080, 630)
      8. TextView: "com.android.settings:id/switch_text", "Use Droidrun Portal" - (79, 494, 534, 564)
      9. Switch: "android:id/switch_widget", "switch_widget" - (863, 483, 1001, 574)
    10. TextView: "android:id/title", "Options" - (78, 719, 1002, 821)
    11. LinearLayout: "LinearLayout" - (0, 821, 1080, 1039)
 

## Live Phone State Monitor

Real-time monitoring of phone state with auto-refresh display (updates every 0.5 seconds)

In [ ]:
from IPython.display import display, clear_output
import time

def get_phone_state() -> Dict[str, Any]:
    """Fetch the current phone state from the device."""
    result = subprocess.run(
        ['adb', 'shell', 'content', 'query', '--uri', 'content://com.droidrun.portal/phone_state'],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        return {"error": True, "message": f"ADB command failed: {result.stderr}"}
    
    for line in result.stdout.strip().split('\n'):
        if 'result=' in line:
            json_str = line.split('result=', 1)[1]
            outer_json = json.loads(json_str)
            
            if outer_json.get('status') != 'success':
                return {"error": True, "message": outer_json.get('error', 'Unknown error')}
            
            # Parse the inner data (which is a JSON string)
            phone_state = json.loads(outer_json['data'])
            return phone_state
    
    return {"error": True, "message": "No result found in ADB output"}

def format_phone_state_display(phone_state: Dict[str, Any]) -> str:
    """Format phone state for clean display."""
    if phone_state.get("error"):
        return f"❌ Error: {phone_state.get('message', 'Unknown error')}"
    
    lines = []
    lines.append("=" * 60)
    lines.append("📱 CURRENT PHONE STATE")
    lines.append("=" * 60)
    lines.append("")
    
    # App info
    current_app = phone_state.get("currentApp", "Unknown")
    package_name = phone_state.get("packageName", "Unknown")
    activity_name = phone_state.get("activityName", "")
    
    lines.append(f"🎯 App: {current_app}")
    lines.append(f"📦 Package: {package_name}")
    
    if activity_name:
        # Extract short activity name (last part after the last dot)
        short_activity = activity_name.split('.')[-1] if '.' in activity_name else activity_name
        lines.append(f"📄 Activity: {short_activity}")
        lines.append(f"   Full: {activity_name}")
    else:
        lines.append(f"📄 Activity: (Not yet captured)")
    
    lines.append("")
    
    # Keyboard status
    is_editable = phone_state.get("isEditable", False)
    keyboard_visible = phone_state.get("keyboardVisible", False)
    keyboard_status = "⌨️  Visible" if is_editable or keyboard_visible else "⌨️  Hidden"
    lines.append(f"Keyboard: {keyboard_status}")
    lines.append("")
    
    # Focused element
    focused_element = phone_state.get("focusedElement", {})
    if focused_element:
        lines.append("🎯 Focused Element:")
        text = focused_element.get("text", "")
        class_name = focused_element.get("className", "")
        resource_id = focused_element.get("resourceId", "")
        
        if text:
            lines.append(f"   Text: '{text}'")
        if class_name:
            lines.append(f"   Class: {class_name}")
        if resource_id:
            lines.append(f"   ID: {resource_id}")
        
        if not text and not class_name and not resource_id:
            lines.append("   (No element focused)")
    else:
        lines.append("🎯 Focused Element: None")
    
    lines.append("")
    lines.append("=" * 60)
    lines.append(f"⏰ Last updated: {time.strftime('%H:%M:%S')}")
    
    return "\n".join(lines)

# Live monitor loop
print("Starting live phone state monitor... (Press interrupt button ■ to stop)")
print()

try:
    while True:
        # Fetch current state
        phone_state = get_phone_state()
        
        # Clear previous output and display new state
        clear_output(wait=True)
        print(format_phone_state_display(phone_state))
        
        # Wait before next update
        time.sleep(0.5)
        
except KeyboardInterrupt:
    clear_output(wait=True)
    print("✅ Live monitor stopped.")
    print(f"Final state at {time.strftime('%H:%M:%S')}:")
    print()
    print(format_phone_state_display(get_phone_state()))

📱 CURRENT PHONE STATE

🎯 App: System UI
📦 Package: com.android.systemui
📄 Activity: MiuiSettings
   Full: com.android.settings.MiuiSettings

Keyboard: ⌨️  Hidden

🎯 Focused Element:
   (No element focused)

⏰ Last updated: 03:45:19
